In [1]:
import openai
import pandas as pd
import numpy as np
import json
import os
from scipy import stats
from scipy.stats import chi2_contingency

In [2]:
os.chdir('/Users/chrissoria/Documents/Research/Categorization_AI_experiments')
current_directory = os.getcwd()
print(current_directory)

/Users/chrissoria/Documents/Research/Categorization_AI_experiments


In [3]:
Main = pd.read_excel("../UCNets_Classification/Hand_Coding_Surveys/a19i/a19i_Master.xlsx", engine='openpyxl', sheet_name="Chris").drop(["Response"], axis=1)
Main = Main.iloc[:400, :8]
Main = Main.drop(['Relationship end','Housing, concretely','New living environment'], axis=1)
Main.columns = range(Main.shape[1])

Claude = pd.read_excel("../UCNets_Classification/Hand_Coding_Surveys/a19i/a19i_Master.xlsx", engine='openpyxl', sheet_name="Claude").drop(["Response","prim_key"], axis=1)
Claude = Claude.iloc[:400, :8]
Claude = Claude.drop(['Relationship end','Housing, concretely','New living environment'], axis=1)
Claude.columns = range(Claude.shape[1])

Rebeca = pd.read_excel("../UCNets_Classification/Hand_Coding_Surveys/a19i/a19i_Master.xlsx", engine='openpyxl', sheet_name="Rebeca").drop(["Response"], axis=1)
Rebeca = Rebeca.iloc[:400, :8]
Rebeca = Rebeca.drop(['Relationship end','Housing, concretely','New living environment'], axis=1)
Rebeca.columns = range(Rebeca.shape[1])

Main

,0,1,2,3,4
0,NaN,NaN,NaN,NaN,1.0
1,1.0,NaN,NaN,NaN,NaN
2,1.0,NaN,NaN,NaN,NaN
3,NaN,1.0,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...
395,NaN,NaN,NaN,NaN,NaN
396,NaN,1.0,NaN,NaN,NaN
397,NaN,NaN,NaN,NaN,NaN
398,NaN,NaN,NaN,1.0,NaN


In [5]:
print(f"The dataframe Main has {len(Main)} rows.")

print(f"The dataframe bad has {len(bad)} rows.")

print(f"The dataframe good has {len(good)} rows.")

print(f"The dataframe cot has {len(cot)} rows.")

print(f"The dataframe cove has {len(cove)} rows.")

print(f"The dataframe oneshot has {len(oneshot)} rows.")

print(f"The dataframe fewshot has {len(fewshot)} rows.")

Main.fillna(0, inplace=True)
Claude.fillna(0, inplace=True)
Rebeca.fillna(0, inplace=True)
bad.fillna(0, inplace=True)
good.fillna(0, inplace=True)
cot.fillna(0, inplace=True)
cove.fillna(0, inplace=True)
oneshot.fillna(0, inplace=True)
fewshot.fillna(0, inplace=True)

The dataframe Main has 400 rows.
The dataframe bad has 400 rows.
The dataframe good has 400 rows.
The dataframe cot has 400 rows.
The dataframe cove has 400 rows.
The dataframe oneshot has 400 rows.
The dataframe fewshot has 400 rows.


In [6]:
Main = Main.astype(int, errors='ignore')
Claude = Claude.astype(int, errors='ignore')
Rebeca = Rebeca.astype(int, errors='ignore')
bad = bad.astype(int, errors='ignore')
good = good.astype(int, errors='ignore')
cot = cot.astype(int, errors='ignore')
cove = cove.astype(int, errors='ignore')
oneshot = oneshot.astype(int, errors='ignore')
fewshet = fewshot.astype(int, errors='ignore')

In [7]:
Main.head()

,0,1,2,3,4
0,0,0,0,0,1
1,1,0,0,0,0
2,1,0,0,0,0
3,0,1,0,0,0
4,0,0,0,0,0


In [8]:
association_measures = []
p_values = []
jaccard_similarities = []
cosine_similarities = []

for i in range(5):
    main_col = Main.columns[i]
    Claude_col = Claude.columns[i]

    # Create a contingency table
    contingency_table = pd.crosstab(Main[main_col], Claude[Claude_col])

    # Calculate the Chi-squared test
    chi2, p_value, _, _ = chi2_contingency(contingency_table)
    association_measures.append(chi2)  # chi2 as a measure of association
    p_values.append(p_value)

    # Calculate Jaccard Similarity
    intersection = sum((Main[main_col] == 1) & (Claude[Claude_col] == 1))
    union = sum((Main[main_col] == 1) | (Claude[Claude_col] == 1))
    jaccard_similarity = intersection / union if union != 0 else 0
    jaccard_similarities.append(jaccard_similarity)

    # Calculate Cosine Similarity
    dot_product = np.dot(Main[main_col], Claude[Claude_col])
    magnitude_main = np.sqrt(np.dot(Main[main_col], Main[main_col]))
    magnitude_Claude = np.sqrt(np.dot(Claude[Claude_col], Claude[Claude_col]))

Claude_df = pd.DataFrame({
    'Chi2_Main_Claude': association_measures,
    'P_value_Main_Claude': p_values,
    'Jaccard_Similarity': jaccard_similarities,
}, index=Main.columns[:5])

print(Claude_df)

   Chi2_Main_Claude  P_value_Main_Claude  Jaccard_Similarity
0        318.673439         2.817454e-71            0.843137
1        340.372203         5.294336e-76            0.900000
2        256.789317         8.597748e-58            0.733333
3        315.907046         1.128374e-70            0.842105
4        257.266268         6.767313e-58            0.714286


In [11]:
category_dfs = {
    'Claude': Claude,
    'Rebeca': Rebeca,
    'bad': bad,
    'good': good,
    'cot': cot,
    'cove': cove,
    'oneshot': oneshot,
    'fewshot': fewshot
}
result_dfs = {}

for cat_name, cat_df in category_dfs.items():
    association_measures = []
    p_values = []
    jaccard_similarities = []

    for i in range(5):
        main_col = Main.columns[i]
        cat_col = cat_df.columns[i]

            # Create a contingency table
        contingency_table = pd.crosstab(Main[main_col], cat_df[cat_col])

            # Calculate the Chi-squared test
        chi2, p_value, _, _ = chi2_contingency(contingency_table)
        association_measures.append(chi2)  # chi2 as a measure of association
        p_values.append(p_value)

            # Calculate Jaccard Similarity
        intersection = sum((Main[main_col] == 1) & (cat_df[cat_col] == 1))
        union = sum((Main[main_col] == 1) | (cat_df[cat_col] == 1))
        jaccard_similarity = intersection / union if union != 0 else 0
        jaccard_similarities.append(jaccard_similarity)

            # Calculate Cosine Similarity
        dot_product = np.dot(Main[main_col], cat_df[cat_col])
        magnitude_main = np.sqrt(np.dot(Main[main_col], Main[main_col]))
        magnitude_cat = np.sqrt(np.dot(cat_df[cat_col], cat_df[cat_col]))

    # Create the DataFrame
    chi_squared_jaccard_cosine_df = pd.DataFrame({
        'Chi2_Main_Cat': association_measures,
        'P_value_Main_Cat': p_values,
        'Jaccard_Similarity': jaccard_similarities,
    }, index=Main.columns[:5])

    # Store the DataFrame in a dictionary with a unique key
    result_dfs[cat_name] = chi_squared_jaccard_cosine_df

In [12]:
result_dfs

{'Claude':    Chi2_Main_Cat  P_value_Main_Cat  Jaccard_Similarity
 0     318.673439      2.817454e-71            0.843137
 1     340.372203      5.294336e-76            0.900000
 2     256.789317      8.597748e-58            0.733333
 3     315.907046      1.128374e-70            0.842105
 4     257.266268      6.767313e-58            0.714286,
 'Rebeca':    Chi2_Main_Cat  P_value_Main_Cat  Jaccard_Similarity
 0     340.810294      4.250144e-76            0.888889
 1     263.388044      3.133441e-59            0.738318
 2     226.751119      3.047129e-51            0.666667
 3     309.980354      2.205486e-69            0.831169
 4     266.305007      7.248314e-60            0.722222,
 'bad':    Chi2_Main_Cat  P_value_Main_Cat  Jaccard_Similarity
 0     270.584109      8.464586e-61            0.733333
 1     298.143335      8.361425e-67            0.824000
 2     237.525477      1.362302e-53            0.687500
 3     195.586974      1.918252e-44            0.596639
 4     190.995347  

In [13]:
# Initialize an empty dictionary to hold the average Jaccard Similarity for each category
average_jaccard_similarities = {}

# Iterate over each item in result_dfs to calculate the average Jaccard Similarity
for cat_name, df in result_dfs.items():
    average_jaccard_similarities[cat_name] = df['Jaccard_Similarity'].mean()

# Convert the dictionary of averages into a DataFrame
# The keys become the column names, and the values become the row values
average_jaccard_df = pd.DataFrame(average_jaccard_similarities, index=["Average_Jaccard_Similarity"])

print(average_jaccard_df)

                              Claude    Rebeca       bad      good      cot  \
Average_Jaccard_Similarity  0.806572  0.769453  0.675987  0.718006  0.74985   

                                cove   oneshot   fewshot  
Average_Jaccard_Similarity  0.687312  0.736034  0.734055  


In [14]:
average_jaccard_df_transposed = average_jaccard_df.T.rename(columns={"Average_Jaccard_Similarity": "Average_Jaccard"})

In [17]:
average_jaccard_df_transposed.to_csv('data/a19i/average_jaccard.csv')

In [16]:
# Initialize an empty DataFrame to hold the Jaccard Similarity columns
jaccard_similarity_df = pd.DataFrame()

# Iterate over the result_dfs dictionary to extract the Jaccard Similarity column
for cat_name, df in result_dfs.items():
    # Add the extracted column to the new DataFrame with a new column name
    jaccard_similarity_df[cat_name] = df['Jaccard_Similarity']

print(jaccard_similarity_df)

     Claude    Rebeca       bad      good       cot      cove   oneshot  \
0  0.843137  0.888889  0.733333  0.800000  0.781818  0.681818  0.785714   
1  0.900000  0.738318  0.824000  0.826772  0.852459  0.846774  0.816000   
2  0.733333  0.666667  0.687500  0.705882  0.705882  0.631579  0.687500   
3  0.842105  0.831169  0.596639  0.700000  0.772727  0.683168  0.765957   
4  0.714286  0.722222  0.538462  0.557377  0.636364  0.593220  0.625000   

    fewshot  
0  0.728814  
1  0.842975  
2  0.666667  
3  0.806818  
4  0.625000  


In [18]:
jaccard_similarity_df.to_csv('data/a19i/similiarity_jaccard.csv')